In [1]:
from datasets import load_dataset
import numpy as np

In [2]:
trainData = load_dataset(
    'cifar100',
    split='train',
    cache_dir="/scratch/doa240/.cache/huggingface/datasets",
    streaming=True,
    ignore_verifications=True  # set to True if seeing splits Error
)


In [3]:
X_train = []
y_train = []

for data in trainData:
    img = data['img']
    img = img.convert('L')
    img = np.array(img)
    X_train.append(img)
    y_train.append(data['fine_label'])

In [4]:
X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0], -1)
print(X_train.shape)

(50000, 1024)


In [5]:
y_train = np.array(y_train)
print(y_train.shape)

(50000,)


In [6]:
import autosklearn.classification
import autosklearn.metrics
from pprint import pprint
import joblib
from smac.tae import StatusType 
from sklearn.metrics import accuracy_score, f1_score 

In [7]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=3600*12, 
    per_run_time_limit = (3600*12)/6,
    seed=101,
    n_jobs=-1
)
automl.fit(X_train, y_train)

/scratch/doa240/conda-envs/pytorch/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45153 instead
  warnings.warn(


[WARNING] [2023-03-25 23:17:18,865:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-03-25 23:17:34,299:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-03-25 23:17:34,592:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-03-25 23:17:34,915:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-03-25 23:17:44,012:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-03-25 23:17:57,636:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-03-25 23:18:07,255:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-03-25 23:18:07,666:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-03-25 23:18:18,919:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2023-03-25 23:18:

init_gesdd failed init
init_gesdd failed init


AutoSklearnClassifier(ensemble_class=<class 'autosklearn.ensembles.ensemble_selection.EnsembleSelection'>,
                      n_jobs=-1, per_run_time_limit=7200.0, seed=101,
                      time_left_for_this_task=43200)

In [8]:
pprint(automl.sprint_statistics())

('auto-sklearn results:\n'
 '  Dataset name: a0237ab2-cb41-11ed-8e64-00110a6b78e8\n'
 '  Metric: accuracy\n'
 '  Best validation score: 0.184061\n'
 '  Number of target algorithm runs: 2655\n'
 '  Number of successful target algorithm runs: 1433\n'
 '  Number of crashed target algorithm runs: 176\n'
 '  Number of target algorithms that exceeded the time limit: 64\n'
 '  Number of target algorithms that exceeded the memory limit: 982\n')


In [9]:
print(automl.leaderboard())

          rank  ensemble_weight type      cost    duration
model_id                                                  
2350         1             0.06  qda  0.815939  320.635545
2443         2             0.02  qda  0.816182  207.804317
1487         3             0.02  qda  0.816242  328.589979
1682         4             0.04  qda  0.816242  310.482866
936          6             0.02  qda  0.816303  452.665245
2544         5             0.06  qda  0.816303  335.336506
851          9             0.02  qda  0.816364  802.501897
1462         7             0.02  qda  0.816364  323.737219
1498         8             0.04  qda  0.816364  316.038868
1153        10             0.02  qda  0.816424  495.375317
1297        15             0.02  qda  0.816424  311.899173
1522        13             0.06  qda  0.816424  314.705023
1537        14             0.04  qda  0.816424  319.264583
1538        12             0.06  qda  0.816424  313.532084
2386        11             0.02  qda  0.816424  198.8638

In [10]:
automl.show_models()

{653: {'model_id': 653,
  'rank': 1,
  'cost': 0.8166666666666667,
  'ensemble_weight': 0.02,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x15540fdb6190>,
  'balancing': Balancing(random_state=101),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x1554900bf850>,
  'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice at 0x1554900bfe50>,
  'sklearn_classifier': QuadraticDiscriminantAnalysis(reg_param=0.6679363870686031)},
 661: {'model_id': 661,
  'rank': 2,
  'cost': 0.8166666666666667,
  'ensemble_weight': 0.06,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x155525572b80>,
  'balancing': Balancing(random_state=101),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x15541695f520>,
  'classifier': <autosklearn.pipeline.component

In [11]:
joblib.dump(automl, 'cifar100.joblib')

['cifar100.joblib']

In [12]:
testData = load_dataset(
    'cifar100',
    split='test',
    streaming=True,
    cache_dir="/scratch/doa240/.cache/huggingface/datasets",
    ignore_verifications=True  # set to True if seeing splits Error
)

In [13]:
X_test = []
y_test = []

for data in testData:
    img = data['img']
    img = img.convert('L')
    img = np.array(img)
    X_test.append(img)
    y_test.append(data['fine_label'])

In [14]:
X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0], -1)
print(X_test.shape)

(10000, 1024)


In [15]:
y_test = np.array(y_test)
print(y_test.shape)

(10000,)


In [16]:
# evaluate
pred = automl.predict(X_test)
test_acc = accuracy_score(y_test, pred)
print("Test Accuracy score {0}".format(test_acc))

Test Accuracy score 0.191


In [19]:
classes = [0] * 100
classesCnt = [0] * 100

for i in range(len(pred)):
    if pred[i] == y_test[i]:
        classes[pred[i]] += 1
    classesCnt[y_test[i]] += 1
    
for i in range(100):
    print("Label:", i, classes[i]/classesCnt[i])

Label: 0 0.42
Label: 1 0.44
Label: 2 0.09
Label: 3 0.14
Label: 4 0.08
Label: 5 0.15
Label: 6 0.05
Label: 7 0.06
Label: 8 0.12
Label: 9 0.39
Label: 10 0.15
Label: 11 0.1
Label: 12 0.07
Label: 13 0.19
Label: 14 0.11
Label: 15 0.05
Label: 16 0.23
Label: 17 0.19
Label: 18 0.0
Label: 19 0.09
Label: 20 0.61
Label: 21 0.32
Label: 22 0.21
Label: 23 0.49
Label: 24 0.67
Label: 25 0.13
Label: 26 0.07
Label: 27 0.1
Label: 28 0.56
Label: 29 0.15
Label: 30 0.1
Label: 31 0.09
Label: 32 0.15
Label: 33 0.17
Label: 34 0.01
Label: 35 0.1
Label: 36 0.14
Label: 37 0.07
Label: 38 0.03
Label: 39 0.11
Label: 40 0.31
Label: 41 0.46
Label: 42 0.04
Label: 43 0.05
Label: 44 0.03
Label: 45 0.1
Label: 46 0.14
Label: 47 0.33
Label: 48 0.35
Label: 49 0.12
Label: 50 0.03
Label: 51 0.11
Label: 52 0.48
Label: 53 0.17
Label: 54 0.07
Label: 55 0.01
Label: 56 0.2
Label: 57 0.18
Label: 58 0.22
Label: 59 0.16
Label: 60 0.76
Label: 61 0.53
Label: 62 0.04
Label: 63 0.19
Label: 64 0.08
Label: 65 0.07
Label: 66 0.01
Label: 67 0.

In [18]:
classes = np.array(classes)
print(np.sum(classes)/len(y_test))

0.191
